In [20]:
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg
import re

import pylab as plt
import seaborn as sns

from statsmodels.stats.multitest import multipletests

/scratch/c.c21013066/docker/envs/timeseries/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Load Data

In [88]:
demo = pd.read_csv('/scratch/c.c21013066/data/ppmi/phenotypes2021/demographics_clean.csv',parse_dates=['date_diagnosis'])
grouped_mean = pd.read_csv('/scratch/c.c21013066/data/ppmi/accelerometer/weekly_mean.csv',parse_dates=['date_y'])
grouped_mean = pd.merge(demo,grouped_mean,on='participant',how='right')
grouped_mean = grouped_mean[grouped_mean['diagnosis']=='pd']

In [89]:
grouped_mean['time_since_diagnosis'] = (grouped_mean['date_y'] -grouped_mean['date_diagnosis'])/np.timedelta64(1,'Y')

In [90]:
print(grouped_mean.groupby('participant').last()['time_since_diagnosis'].describe())
grouped_mean.groupby('participant').last()['visit_age'].describe()

count    85.000000
mean      6.813910
std       2.107513
min       2.499709
25%       4.999418
50%       7.165103
75%       8.413588
max      11.666222
Name: time_since_diagnosis, dtype: float64


count    85.000000
mean     67.688599
std       7.543153
min      46.245987
25%      62.752829
50%      68.420296
75%      73.249964
max      89.918342
Name: visit_age, dtype: float64

In [91]:
# how many have more than 1 visit?
n_visits = grouped_mean.groupby('participant').size().rename('size')
grouped_mean = pd.merge(n_visits,grouped_mean,on='participant',how='outer')
grouped_mean = grouped_mean[grouped_mean['size']>1]
n_visits.value_counts()

In [95]:
neuropsychiatric = ['stai_trait','stai_state','gds','quip']
cognition = ['semantic_fluency','moca','benton','lns','hvlt_recall','hvlt_recognition','hvlt_retention','symbol_digit']
autonome = ['epworth','rbd','systolic_bp_drop','scopa_aut']
daily = ['se_adl','updrs_i']
motor = ['updrs_ii','updrs_iii_OFF']
medication = ['updrs_iii_ON','updrs_iv','LEDD',]
features = np.hstack([cognition,neuropsychiatric,autonome,daily,motor,medication])
covs = ['visit_age','date_y']
predictors = grouped_mean.filter(regex='(walking|step|efficiency|total_sleep|pulse|deep|light|rem|nrem|rmssd|wake)').columns
sleep_col = grouped_mean.filter(regex='(efficiency|total_sleep|deep|light|rem|nrem|wake)').columns
phys = grouped_mean.filter(regex='(walking|step)').columns
vital = grouped_mean.filter(regex='(pulse|rmssd)').columns
predictors_filt = [a for a in predictors if not re.search('_ms', a)]
sleep_col = [a for a in sleep_col if not re.search('_ms', a)]
phys = [a for a in phys if not re.search('_ms', a)]
vital = [a for a in vital if not re.search('_ms', a)]

# Sensitivity to detect change
cohen's d between first and second visit

In [96]:
first_visit = grouped_mean.groupby('participant').first()
second_visit = grouped_mean.groupby('participant').nth(1)

visit_diff = first_visit[np.hstack([features,predictors_filt])] - second_visit[np.hstack([features,predictors_filt])]

In [97]:
((second_visit['date_y'] - first_visit['date_y'])/np.timedelta64(1,'Y')).agg(['mean','std','size'])

mean     0.629484
std      0.222128
size    35.000000
Name: date_y, dtype: float64

In [98]:
first_visit['time_since_diagnosis'].agg(['mean','std'])

mean    5.546843
std     2.314049
Name: time_since_diagnosis, dtype: float64

In [100]:
cohen = pd.DataFrame(columns=['T', 'dof', 'alternative', 'p-val', 'CI95%', 'cohen-d', 'BF10', 'power','hedges g','cohen'],index=np.hstack([features,predictors_filt]))
for test in np.hstack([features,predictors_filt]):
    cohen.loc[test,['T', 'dof', 'alternative', 'p-val', 'CI95%', 'cohen-d', 'BF10', 'power']] = pg.ttest(second_visit[test],first_visit[test],correction=False,paired=True).values
    cohen.loc[test,['hedges g']] = np.abs(pg.compute_effsize(first_visit[test],second_visit[test],paired=True,eftype='hedges'))

In [101]:
cohen = cohen.reset_index()
cohen['category'] = cohen['index'].apply(lambda x: 'digital' if x in predictors_filt else 'clinical')
cohen['cohen-d'] = cohen['cohen-d'].astype(float)
cohen['hedges g'] = cohen['hedges g'].astype(float)
cohen = cohen.set_index('index')

In [106]:
print(cohen.groupby('category').mean())
pg.ttest(cohen.loc[cohen['category']=='clinical',"cohen-d"],cohen.loc[cohen['category']=='digital',"cohen-d"],correction=False)

           cohen-d  hedges g
category                    
clinical  0.129012  0.127059
digital   0.246596  0.240368


/tmp/ipykernel_176695/2512666005.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(cohen.groupby('category').mean())


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.78894,35,two-sided,0.008496,"[-0.2, -0.03]",0.945392,5.665,0.773898


In [103]:
cohen.to_csv('/scratch/c.c21013066/data/ppmi/analyses/studywatch/visit12_difference_cohen.csv')

# Associations of change over time

In [53]:
# compute difference between visits
diff = grouped_mean.groupby('participant')[np.hstack([features,predictors_filt,covs])].diff()
grouped_mean[[f'{name}_diff' for name in np.hstack([features,predictors_filt,covs])]] = diff
# scale by time_diff
grouped_mean[[f'{name}_diff' for name in np.hstack([features,predictors_filt])]] = grouped_mean[[f'{name}_diff' for name in np.hstack([features,predictors_filt])]].div(grouped_mean['visit_age_diff'],
axis=0)
# average over visits
diff_mean = grouped_mean.groupby('participant').mean()#nth(1)

     sum_walking_minutes_diff  visit_age_diff date_y_diff
1                         NaN             NaN         NaT
2                    0.000000        0.837800    306 days
8                         NaN             NaN         NaT
9                    0.888889        0.919937    336 days
11                        NaN             NaN         NaT
..                        ...             ...         ...
128                       NaN             NaN         NaT
129                 -2.197040        0.501037    183 days
130                       NaN             NaN         NaT
131                 -0.706633        0.503775    184 days
132                 -0.833333        1.002074    366 days

[84 rows x 3 columns]
     sum_walking_minutes_diff  visit_age_diff date_y_diff
1                         NaN             NaN         NaT
2                    0.000000        0.837800    306 days
8                         NaN             NaN         NaT
9                    0.966250        0.919937    

In [57]:
cl_names = ['Semantic Fluency', 'MOCA', 'Benton',
       'Letter Number Sequencing', 'HVLT Recall', 'HVLT Recognition', 'HVLT Retention',
       'Symbol Digit', 'STAI trait', 'STAI state', 'GDS', 'QUIP',
       'ESS', 'RBDSQ', 'Systolic BP Drop', 'SCOPA autonome',
       'Schwab England ADL', 'UPDRS I','UPDRS II','UPDRS III OFF','UPDRS III ON','UPDRS IV', 'LEDD']

In [59]:
corr = pd.DataFrame(columns=['N','pearson r','p-value'],index=pd.MultiIndex.from_product([np.hstack([cl_names,predictors_filt]),
                                                                                          np.hstack([cl_names,predictors_filt])],
                                                                                         names=['f1','f2']))
for (i,p),m in zip(enumerate([f'{name}_diff' for name in np.hstack([features,predictors_filt])]),
                               np.hstack([cl_names,predictors_filt])):
    for (j,u),n in zip(enumerate([f'{name}_diff' for name in np.hstack([features,predictors_filt])]),
                               np.hstack([cl_names,predictors_filt])):
        if j>i:
            try:
                dat = diff_mean.dropna(subset=[p,u])
                corr.loc[(m,n),'N'] = dat.shape[0]
                r,pval = stats.pearsonr(dat[p],dat[u])
                corr.loc[(m,n),'pearson r'] = r
                corr.loc[(m,n),'p-value'] = pval
            except:
                print(m,n)

Semantic Fluency sleep_efficiency
Semantic Fluency num_awakenings
Semantic Fluency total_sleep_time_hour
Semantic Fluency wake_after_sleep_onset_hour
Semantic Fluency total_nrem_time_hour
Semantic Fluency total_rem_time_hour
Semantic Fluency total_deep_nrem_time_hour
Semantic Fluency total_light_nrem_time_hour
MOCA sleep_efficiency
MOCA num_awakenings
MOCA total_sleep_time_hour
MOCA wake_after_sleep_onset_hour
MOCA total_nrem_time_hour
MOCA total_rem_time_hour
MOCA total_deep_nrem_time_hour
MOCA total_light_nrem_time_hour
Benton sleep_efficiency
Benton num_awakenings
Benton total_sleep_time_hour
Benton wake_after_sleep_onset_hour
Benton total_nrem_time_hour
Benton total_rem_time_hour
Benton total_deep_nrem_time_hour
Benton total_light_nrem_time_hour
Letter Number Sequencing sleep_efficiency
Letter Number Sequencing num_awakenings
Letter Number Sequencing total_sleep_time_hour
Letter Number Sequencing wake_after_sleep_onset_hour
Letter Number Sequencing total_nrem_time_hour
Letter Numbe

In [31]:
corr.to_csv('/scratch/c.c21013066/data/ppmi/analyses/studywatch/ratechange__month_mean_corr.csv')